# Piecewise Pricing for Revenue Maximization

[![Investment_project.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_2.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_2.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_2.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_2.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_2.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_2.ipynb)](https://colab.ampl.com)

This model finds the point that maximizes profit based on price elasticity and cost.


[*Partner with the AMPL team to transform complex problems into optimized solutions. AMPL consulting services combine deep technical knowledge with industry-leading insights, helping you unlock the full potential of optimization within your organization.*](https://ampl.com/services/)

Tags: Lessons, ampl, MIP, 

Notebook author: Mikhail Riabtsev <<mail@solverytic.com>>
***

# 1. Download Necessary Extensions and Libraries

In [4]:
# Install dependencies
%pip install -q amplpy pandas
import pandas as pd                 # Loading panda to work with pandas.DataFrame objects (https://pandas.pydata.org/)
import numpy as np                  # Loading numpy to perform multidimensional calculations numpy.matrix (https://numpy.org/)

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cbc", "highs", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

'# Google Colab & Kaggle integration\nfrom amplpy import AMPL, ampl_notebook\n\nampl = ampl_notebook(\n    modules=["cbc", "highs", "gurobi"],  # modules to install\n    license_uuid="default",  # license to use\n)  # instantiate AMPL object and register magics'

# 2. Model description

This model focuses on optimizing revenue for a product with a piecewise linear price-demand relationship. The objective is to determine the optimal price step and corresponding quantity sold to maximize revenue while considering production and demand constraints. The model is particularly suitable for businesses dealing with products that have a nonlinear demand curve influenced by pricing strategies.

## Key Features:
### Piecewise Linear Pricing:
- The demand curve is divided into discrete price steps, each associated with a specific demand level and price.
- Only one price step can be selected in the final solution.

### Revenue Maximization:
- Revenue is calculated as the product of demand and price, adjusted for production costs and unsold inventory.

### Quantity Constraints:
- The total quantity sold must not exceed the available inventory.
- Quantities sold at each step are bounded by the price-demand relationships to ensure logical consistency.

### Binary Decision Variables:
- A binary variable ensures that only one price step is chosen, simplifying decision-making and aligning with practical business scenarios.

## Parameters:
- ***$total\_Quantity$***: Total available inventory for sale.
- ***$unit\_Cost$***: Cost of producing one unit of the product.
- ***$nStep$***: Number of discrete price steps in the demand curve.
- ***$demand[i]$***: Expected demand at each price step.
- ***$price[i]$***: Price corresponding to each demand value.

## Variables:

- ***$Quantity\_Sold[i]$***: Quantity sold at each price step.
- ***$Price\_Selected[i]$***: Binary variable indicating whether a price step is chosen.

## Objective:
Maximize the total revenue by selecting the optimal price step and determining the corresponding quantity sold, subject to production and demand constraints.

$Maximize:$
$∑_{𝑖=1}^{𝑛𝑆𝑡𝑒𝑝}(demand[𝑖]⋅price[𝑖]⋅Price\_Selected[𝑖]−(Quantity\_Sold[𝑖]−demand[𝑖]⋅Price\_Selected[𝑖])−Quantity\_Sold[𝑖]⋅unit\_Cost)$

## Constraints:
- ***Single Price Step Selection***: Only one price step can be chosen.

   $∑_{𝑖=1}^{𝑛𝑆𝑡𝑒𝑝}Price\_Selected[𝑖]=1$

- ***Total Quantity Limit***: Total quantity sold cannot exceed available inventory.

   $∑_{𝑖=1}^{𝑛𝑆𝑡𝑒𝑝}Quantity\_Sold[𝑖]≤total\_Quantity$

***Price Step Bounds:***
      
- ***Upper Bound:*** Quantity sold must be at least the demand of the selected price step.
      
   $Quantity\_Sold[i]≥demand[i]⋅Price\_Selected[i]$
   
- ***Lower Bound:*** Quantity sold must not exceed the demand of the next price step.

   $Quantity\_Sold[i]≤demand[i+1]⋅Price\_Selected[i]$

# 3. AMPL Model Formulation

In [7]:
%%writefile demand_elasticity_2_model.mod
reset;
### PARAMETERS
param total_Quantity >= 0;          # Total quantity of products available for sale
param unit_Cost >= 0;               # Unit cost of the product

param nStep integer > 0;            # Number of steps in the piecewise linear price function
param demand {1..nStep+1} >= 0;     # Demand values at each step
param price {1..nStep+1} >= 0;      # Price Price of demand for each step


### VARIABLES
var Quantity_Sold {1..nStep} >= 0 ; # Quantity sold at each price step
var Price_Selected {1..nStep} binary;# Binary variable indicating whether the price step is selected


### OBJECTIVE
maximize Total_Revenue:             # Maximize total revenue (profit from sales)
    sum {i in 1..nStep} (
        demand[i] * price[i] * Price_Selected[i] 
        - (Quantity_Sold[i] - demand[i] * Price_Selected[i])
        - Quantity_Sold[i] * unit_Cost);


### CONSTRAINTS
s.t. Single_Price:                  # Only one price step can be selected
    sum {i in 1..nStep} Price_Selected[i] = 1;

s.t. Quantity_Limit:                # The total quantity sold cannot exceed available quantity
    sum {i in 1..nStep} Quantity_Sold[i] <= total_Quantity;

s.t. Price_Upper_Bound {i in 1..nStep}: # Quantity sold cannot exceed upper price step
    Quantity_Sold[i] >= demand[i] * Price_Selected[i];

s.t. Price_Lower_Bound {i in 1..nStep}: # Quantity sold must be at least the next price step
    Quantity_Sold[i] <= demand[i+1] * Price_Selected[i];

Overwriting demand_elasticity_2_model.mod


# 4. Load data

In [12]:
ampl.read('demand_elasticity_3_model.mod')                        # Load the AMPL model from the file

# Set global parameters for the model
ampl.param['total_Quantity'] = 600                                # Total available quantity for sale
ampl.param['unit_Cost'] = 2                                       # Unit cost of the product
ampl.param['nStep'] = 13                                          # Number of price steps

# Define the data points as (demand, price) pairs for each step
data = {
   1: (0, 10), 
   2: (1, 8), 
   3: (5, 7.5),
   4: (10, 7),
   5: (15, 6.5),
   6: (30, 6),
   7: (60, 5.5),
   8: (100, 5),
   9: (150, 4.5),
   10: (200, 4),
   11: (240, 3.5),
   12: (270, 3),
   13: (280, 2.5),
   14: (290, 2)}

# Set the demand and price parameters in AMPL
demand_param = ampl.getParameter("demand")                        # Get the 'demand' parameter
price_param = ampl.getParameter("price")                          # Get the 'price' parameter
for i, (demand_value, price_value) in data.items():               # Loop through data points
    demand_param.set(i, demand_value)                             # Assign demand value to each step
    price_param.set(i, price_value)                               # Assign price value to each step

# 5. Solve problem

In [9]:
# Set the solver type for use in solving the problems
solver = 'cplex'  # Use CBC solver for optimization tasks

ampl.option['show_stats'] = 0 # Show problem size statistics (default: 0)
ampl.option['display_1col'] = 0 # Disable single-column data display
#ampl.option['omit_zero_rows'] = 1 # Hide rows with zero values
#ampl.option['omit_zero_cols'] = 1 # Hide columns with zero values
ampl.option['mp_options'] = 'outlev=1 lim:time=20'   # Configure CBC options (output level and time limit)

ampl.solve(solver=solver, verbose=False)   # Solve the optimization problem using CBC solver  

## 6. Display results

In [10]:
# Display results for key variables
ampl.display('_varname', '_var', '_var.lb', '_var.ub', '_var.rc', '_var.slack')
ampl.display('_conname', '_con', '_con.body', '_con.lb', '_con.ub', '_con.slack')
ampl.display('_objname', '_obj')

:          _varname        _var _var.lb   _var.ub   _var.rc _var.slack    :=
1    'Quantity_Sold[1]'       0     0     Infinity     -3         0
2    'Quantity_Sold[2]'       0     0     Infinity     -3         0
3    'Quantity_Sold[3]'       0     0     Infinity     -3         0
4    'Quantity_Sold[4]'       0     0     Infinity     -3         0
5    'Quantity_Sold[5]'       0     0     Infinity     -3         0
6    'Quantity_Sold[6]'       0     0     Infinity     -3         0
7    'Quantity_Sold[7]'       0     0     Infinity     -3         0
8    'Quantity_Sold[8]'       0     0     Infinity     -3         0
9    'Quantity_Sold[9]'       0     0     Infinity     -3         0
10   'Quantity_Sold[10]'    200     0     Infinity      0       200
11   'Quantity_Sold[11]'      0     0     Infinity     -3         0
12   'Quantity_Sold[12]'      0     0     Infinity     -3         0
13   'Quantity_Sold[13]'      0     0     Infinity     -3         0
14   'Price_Selected[1]'      0     0  

# 7. Retrieve solution in Python

In [11]:
# Initialize an empty dictionary to store AMPL variable data
amplvar = dict()

# Prepare a list of AMPL variables
list_of_ampl_variables = [item[0] for item in ampl.get_variables()]

# Iterate over each variable name in the list
for key_ampl in list_of_ampl_variables:
    # Skip certain variables that are not to be processed (these variables won't be included in the output)
    if key_ampl not in ['']:
        # Convert the AMPL variable data to a pandas DataFrame
        df = ampl.var[key_ampl].to_pandas()
        # Filter the DataFrame to include only rows where the variable's value is greater than a small threshold (1e-5)
        filtered_df = df[df[f"{key_ampl}.val"] > 1e-5]
        # Round the values in the DataFrame to two decimal places
        rounded_df = filtered_df.round(2)
        # Convert the filtered DataFrame to a dictionary and add it to the amplvar dictionary
        amplvar[key_ampl] = rounded_df #.to_dict(orient='records')
print (amplvar[key_ampl])

    Quantity_Sold.val
10                200
